### Random Forest XgBoost
- Split FINAL_DATASET for train and test
- Run regression for random forest w/ different learning rates and depth
- Use CV to cross-validate the features
- Run F-Score to see feature importance (see Stephan ipynb file)

In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

final_csv = pd.read_csv('../Preprocessing/Data_Structures/FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_GOALS_ADDED.csv')
final_csv['home_shot_success_rate'] = 0
final_csv['away_shot_success_rate'] = 0
final_csv = final_csv.drop('Unnamed: 0', axis=1)

#print final_csv[3, 'home_team_api_id']

for index in final_csv.index:
    home_total_shots = (final_csv.loc[index,'shoton_home']+final_csv.loc[index,'shotoff_home'])
    away_total_shots = (final_csv.loc[index,'shoton_away']+final_csv.loc[index,'shotoff_away'])
    if (home_total_shots == 0):
        final_csv.loc[index, 'home_shot_success_rate'] = 0
    else:
        final_csv.loc[index, 'home_shot_success_rate'] = final_csv.loc[index,'home_team_goal']/home_total_shots
    if (away_total_shots == 0):
        final_csv.loc[index, 'away_shot_success_rate'] = 0
    else:
        final_csv.loc[index, 'away_shot_success_rate'] = final_csv.loc[index, 'away_team_goal']/away_total_shots

final_csv.head()

,away_Defender_rating,home_Defender_rating,away_Attacker_rating,home_Attacker_rating,away_Goalkeeper_rating,home_Goalkeeper_rating,away_Midfielder_rating,home_Midfielder_rating,HEAD_2_HEAD_HOME_TEAM_WINS,HEAD_2_HEAD_HOME_TEAM_LOSS,...,BWD,BWA,IWH,IWD,IWA,home_team_goal,away_team_goal,Label,home_shot_success_rate,away_shot_success_rate
0,64.500000,72.666667,67.666667,74.0,63.0,72.0,64.333333,73.833333,1.000,0.0,...,4.90,10.00,1.25,4.5,9.0,2,0,2.0,0.0,0.0
1,64.600000,65.666667,66.500000,69.0,69.0,71.0,66.000000,69.600000,1.000,0.0,...,3.60,4.60,1.65,3.3,4.4,3,1,2.0,0.0,0.0
2,64.000000,64.333333,67.000000,66.0,63.0,69.0,65.400000,65.600000,0.000,0.0,...,3.30,3.10,2.20,3.1,3.0,1,1,1.0,0.0,0.0
3,63.000000,64.750000,68.500000,65.0,71.0,65.0,69.666667,65.500000,0.000,1.0,...,3.10,2.35,2.50,3.3,2.4,0,1,0.0,0.0,0.0
4,64.666667,61.000000,67.333333,0.0,68.0,58.0,65.250000,61.000000,0.125,0.5,...,3.15,2.40,2.70,3.2,2.3,0,1,0.0,0.0,0.0


In [49]:
final_csv.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17369 entries, 0 to 17368
Data columns (total 61 columns):
away_Defender_rating             15767 non-null float64
home_Defender_rating             15767 non-null float64
away_Attacker_rating             15767 non-null float64
home_Attacker_rating             15767 non-null float64
away_Goalkeeper_rating           15767 non-null float64
home_Goalkeeper_rating           15767 non-null float64
away_Midfielder_rating           15767 non-null float64
home_Midfielder_rating           15767 non-null float64
HEAD_2_HEAD_HOME_TEAM_WINS       15767 non-null float64
HEAD_2_HEAD_HOME_TEAM_LOSS       15767 non-null float64
HEAD_2_HEAD_DRAW                 15767 non-null float64
HOME_WIN_RATE                    15767 non-null float64
HOME_WIN_RATE_THIS_SEASON        15767 non-null float64
AWAY_WIN_RATE                    15767 non-null float64
AWAY_WIN_RATE_THIS_SEASON        15767 non-null float64
AWAY_WIN_RATE_AT_THIS_GROUND     15767 non-null flo